# Problem

Interpret a toy model trained to take the minimum of two ints.

# Setup
(No need to read)

In [13]:
!git clone https://github.com/ckkissane/mech-interp-practice.git

fatal: destination path 'mech-interp-practice' already exists and is not an empty directory.


In [14]:
# Janky code to do different setup when run in a Colab notebook vs VSCode
DEBUG_MODE = False
try:
    import google.colab
    IN_COLAB = True
    print("Running as a Colab notebook")
    %pip install git+https://github.com/neelnanda-io/TransformerLens.git
    # Install another version of node that makes PySvelte work way faster
    !curl -fsSL https://deb.nodesource.com/setup_16.x | sudo -E bash -; sudo apt-get install -y nodejs
    %pip install git+https://github.com/neelnanda-io/PySvelte.git
    # Needed for PySvelte to work, v3 came out and broke things...
    %pip install typeguard==2.13.3
except:
    IN_COLAB = False
    print("Running as a Jupyter notebook - intended for development only!")
    from IPython import get_ipython

    ipython = get_ipython()
    # Code to automatically update the HookedTransformer code as its edited without restarting the kernel
    ipython.magic("load_ext autoreload")
    ipython.magic("autoreload 2")

Running as a Colab notebook
  Cloning https://github.com/neelnanda-io/TransformerLens.git to /tmp/pip-req-build-ivjlcxgj
  Running command git clone --filter=blob:none --quiet https://github.com/neelnanda-io/TransformerLens.git /tmp/pip-req-build-ivjlcxgj
  Resolved https://github.com/neelnanda-io/TransformerLens.git to commit 10d2f8a026d73eada861c7d51064f7e24d8f482c
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done

## Installing the NodeSource Node.js 16.x repo...


## Populating apt-get cache...

+ apt-get update
Hit:1 https://deb.nodesource.com/node_16.x jammy InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-up

In [15]:
# Plotly needs a different renderer for VSCode/Notebooks vs Colab argh
import plotly.io as pio

if IN_COLAB or not DEBUG_MODE:
    # Thanks to annoying rendering issues, Plotly graphics will either show up in colab OR Vscode depending on the renderer - this is bad for developing demos! Thus creating a debug mode.
    pio.renderers.default = "colab"
else:
    pio.renderers.default = "png"

In [16]:
# Import stuff
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import einops
import tqdm.notebook as tqdm
import random
from pathlib import Path
import plotly.express as px
from torch.utils.data import DataLoader

from jaxtyping import Float, Int
from typing import List, Union, Optional
from functools import partial
import copy

import itertools
from transformers import AutoModelForCausalLM, AutoConfig, AutoTokenizer
import dataclasses
from dataclasses import dataclass
import datasets
from IPython.display import HTML

In [17]:
import pysvelte

import transformer_lens
import transformer_lens.utils as utils
from transformer_lens.hook_points import (
    HookedRootModule,
    HookPoint,
)  # Hooking utilities
from transformer_lens import HookedTransformer, HookedTransformerConfig, FactoredMatrix, ActivationCache

Plotting helper functions:

In [18]:
import plotly.graph_objects as go

update_layout_set = {"xaxis_range", "yaxis_range", "hovermode", "xaxis_title", "yaxis_title", "colorbar", "colorscale", "coloraxis", "title_x", "bargap", "bargroupgap", "xaxis_tickformat", "yaxis_tickformat", "title_y", "legend_title_text", "xaxis_showgrid", "xaxis_gridwidth", "xaxis_gridcolor", "yaxis_showgrid", "yaxis_gridwidth"}
def imshow(tensor, renderer=None, xaxis="", yaxis="", **kwargs):
    kwargs_post = {k: v for k, v in kwargs.items() if k in update_layout_set}
    kwargs_pre = {k: v for k, v in kwargs.items() if k not in update_layout_set}
    if "facet_labels" in kwargs_pre:
        facet_labels = kwargs_pre.pop("facet_labels")
    else:
        facet_labels = None
    if "color_continuous_scale" not in kwargs_pre:
        kwargs_pre["color_continuous_scale"] = "RdBu"
    fig = px.imshow(utils.to_numpy(tensor), color_continuous_midpoint=0.0,labels={"x":xaxis, "y":yaxis}, **kwargs_pre).update_layout(**kwargs_post)
    if facet_labels:
        for i, label in enumerate(facet_labels):
            fig.layout.annotations[i]['text'] = label

    fig.show(renderer)

def line(tensor, renderer=None, xaxis="", yaxis="", **kwargs):
    px.line(y=utils.to_numpy(tensor), labels={"x":xaxis, "y":yaxis}, **kwargs).show(renderer)

def scatter(x, y, xaxis="", yaxis="", caxis="", renderer=None, **kwargs):
    x = utils.to_numpy(x)
    y = utils.to_numpy(y)
    px.scatter(y=y, x=x, labels={"x":xaxis, "y":yaxis, "color":caxis}, **kwargs).show(renderer)

def lines(lines_list, x=None, mode='lines', labels=None, xaxis='', yaxis='', title = '', log_y=False, hover=None, **kwargs):
    # Helper function to plot multiple lines
    if type(lines_list)==torch.Tensor:
        lines_list = [lines_list[i] for i in range(lines_list.shape[0])]
    if x is None:
        x=np.arange(len(lines_list[0]))
    fig = go.Figure(layout={'title':title})
    fig.update_xaxes(title=xaxis)
    fig.update_yaxes(title=yaxis)
    for c, line in enumerate(lines_list):
        if type(line)==torch.Tensor:
            line = utils.to_numpy(line)
        if labels is not None:
            label = labels[c]
        else:
            label = c
        fig.add_trace(go.Scatter(x=x, y=line, mode=mode, name=label, hovertext=hover, **kwargs))
    if log_y:
        fig.update_layout(yaxis_type="log")
    fig.show()

def bar(tensor, renderer=None, xaxis="", yaxis="", **kwargs):
    px.bar(
        y=utils.to_numpy(tensor),
        labels={"x": xaxis, "y": yaxis},
        template="simple_white",
        **kwargs).show(renderer)

In [19]:
import transformer_lens.patching as patching
from transformer_lens import evals
import math

In [20]:
def visualize_attn_patterns(heads, local_tokens, local_cache, title: str = ""):
    labels = []
    patterns = []
    batch_index = 0

    for head in heads:
        if isinstance(head, tuple):
            layer, head_index = head
        else:
            layer, head_index = head // model.cfg.n_heads, head % model.cfg.n_heads
        patterns.append(local_cache["pattern", layer][batch_index, head_index])
        labels.append(f"L{layer}H{head_index}")
    patterns = torch.stack(patterns, dim=-1)
    attn_viz = pysvelte.AttentionMulti(tokens=model.to_str_tokens(local_tokens[batch_index]), attention=patterns, head_labels=labels)
    display(HTML(f"<h3>{title}</h3>"))
    attn_viz.show()

# Load Model

In [21]:
torch.set_grad_enabled(False)

In [22]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


The simplest model I found that can learn this task is a 1L, 1 head, attn-only transformer with no biases, layernorms, or positional embeddings.

The model has already been trained seperately, and is loaded into this notebook below:

In [23]:
MAX_NUM=200
cfg = HookedTransformerConfig(
    n_layers=1,
    n_heads=1,
    d_model=32,
    d_head=32,
    n_ctx=3, # a b =
    d_vocab=MAX_NUM+1, # 0, 1, ..., MAX_NUM-1, =
    d_vocab_out=MAX_NUM,
    normalization_type=None,
    attn_only=True,
    device=device,
    seed=0
)

model = HookedTransformer(cfg)
print(model)

HookedTransformer(
  (embed): Embed()
  (hook_embed): HookPoint()
  (pos_embed): PosEmbed()
  (hook_pos_embed): HookPoint()
  (blocks): ModuleList(
    (0): TransformerBlock(
      (ln1): Identity()
      (attn): Attention(
        (hook_k): HookPoint()
        (hook_q): HookPoint()
        (hook_v): HookPoint()
        (hook_z): HookPoint()
        (hook_attn_scores): HookPoint()
        (hook_pattern): HookPoint()
        (hook_result): HookPoint()
      )
      (hook_q_input): HookPoint()
      (hook_k_input): HookPoint()
      (hook_v_input): HookPoint()
      (hook_attn_out): HookPoint()
      (hook_mlp_in): HookPoint()
      (hook_mlp_out): HookPoint()
      (hook_resid_pre): HookPoint()
      (hook_resid_post): HookPoint()
    )
  )
  (unembed): Unembed()
)


In [24]:
filename = "mech-interp-practice/models/min_int_model.pt"
state_dict = torch.load(filename)

state_dict = torch.load(filename)
model.load_state_dict(state_dict, strict=False);

# Task description

Given a sequence [a, b, =] the model was trained to output min(a,b) at the '=' position. Both $a$ and $b$ are integers which range between [0, 199] inclusive. The special '=' token (200) is always at the end of every sequence.


Below I provide the same dataset I used to train the model.


* 'dataset' is the full dataset - every possible [a, b, =] example for $a, b \in$ [0, 199] inclusive
* 'labels' are the corresponding labels: min(a,b) for each [a,b] pair in 'dataset'
* 'train_data' is a subset of 'dataset' used to train the model. 'train_labels' are the corresponding labels
* 'test_data' and 'test_labels' are the remaining 'dataset' examples and corresponding labels. These were held out during training, and just used to evaluate the model.

In [25]:
a_vec = einops.repeat(torch.arange(MAX_NUM), "i -> (i j)", j=MAX_NUM)
print(a_vec.shape)
print(a_vec[:5])

b_vec = einops.repeat(torch.arange(MAX_NUM), "i -> (j i)", j=MAX_NUM)
print(b_vec.shape)
print(b_vec[:5])

eq_vec = torch.ones_like(a_vec) * cfg.d_vocab-1
print(eq_vec.shape)
print(eq_vec[:5])

dataset = torch.stack([a_vec, b_vec, eq_vec], dim=-1).to(device)
print(dataset.shape)
print(dataset[:5])

torch.Size([40000])
tensor([0, 0, 0, 0, 0])
torch.Size([40000])
tensor([0, 1, 2, 3, 4])
torch.Size([40000])
tensor([200, 200, 200, 200, 200])
torch.Size([40000, 3])
tensor([[  0,   0, 200],
        [  0,   1, 200],
        [  0,   2, 200],
        [  0,   3, 200],
        [  0,   4, 200]], device='cuda:0')


In [26]:
labels = dataset.min(dim=-1).values
print(labels.shape)
print(labels[:5])

torch.Size([40000])
tensor([0, 0, 0, 0, 0], device='cuda:0')


In [27]:
DATA_SEED = 4
torch.manual_seed(DATA_SEED)

train_frac = 0.8

indices = torch.randperm(dataset.shape[0])
cutoff = int(train_frac * dataset.shape[0])

train_indices = indices[:cutoff]
test_indices = indices[cutoff:]

train_data = dataset[train_indices]
train_labels = labels[train_indices]
print(train_data.shape)
print(train_data[:5])
print(train_labels.shape)
print(train_labels[:5])

test_data = dataset[test_indices]
test_labels = labels[test_indices]
print(test_data.shape)
print(test_data[:5])
print(test_labels.shape)
print(test_labels[:5])

torch.Size([32000, 3])
tensor([[  7, 130, 200],
        [182,   0, 200],
        [110, 155, 200],
        [ 82,   8, 200],
        [126,  97, 200]], device='cuda:0')
torch.Size([32000])
tensor([  7,   0, 110,   8,  97], device='cuda:0')
torch.Size([8000, 3])
tensor([[ 19, 171, 200],
        [  6, 195, 200],
        [ 77,   0, 200],
        [ 97,  39, 200],
        [194,  42, 200]], device='cuda:0')
torch.Size([8000])
tensor([19,  6,  0, 39, 42], device='cuda:0')


The model was trained to minimize cross entropy loss. I provide the exact loss function I used during training here:

In [28]:
def loss_fn(logits, labels):
    if logits.ndim == 3:
        logits = logits[:, -1, :]

    logits = logits.to(torch.float64)
    log_probs = logits.log_softmax(dim=-1)
    correct_log_probs = log_probs.gather(dim=-1, index=labels[..., None])
    return -correct_log_probs.mean()